In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np

import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.colors import ListedColormap
import webcolors
from PIL import Image, ImageDraw

import networkx as nx

from minimal.imaging import blit_rooms, draw_sep_nask_wireframe
from minimal.layout import NodeType, NODE_COLOR
from minimal.gen import PlanMasks
from minimal.rooms import RoomAreas, extract_rooms, scale_room_mask
from minimal.walls import create_sep_mask, scale_sep_mask, _conv_mask
from minimal.walls import (
    CC_TL,
    CC_TR,
    CC_BR,
    CC_BL,
    CC_T,
    CC_R,
    CC_B,
    CC_L
)
from minimal.doors import extract_face_walls, candidate_wall_runs

%matplotlib inline

In [4]:
cmapr = ListedColormap(['white', *plt.get_cmap('tab10').colors])
cmapw = ListedColormap(['white', 'black', *plt.get_cmap('tab10').colors])

In [5]:
%%time
pm = PlanMasks.create_from_state(torch.load("data/plan_masks_05.pth"))
rooms = extract_rooms(pm)
rooms.sort(key=lambda r: r.total_area())
# blit_rooms(rooms)

CPU times: user 813 ms, sys: 0 ns, total: 813 ms
Wall time: 812 ms


In [6]:
room_mask = sum(room.to_mask() * (i + 1) for i, room in enumerate(rooms))
sep_mask = create_sep_mask(room_mask)

In [252]:
sx = 3
sy = 3

scaled_sep_mask = scale_sep_mask(sep_mask, sx, sy)
scaled_room_mask = scale_room_mask(room_mask, sx, sy)

inv_room_mask = [
    (scaled_room_mask != i).byte() for i in range(1, len(rooms) + 1)
]

In [8]:
face_walls = extract_face_walls(scaled_sep_mask)

In [323]:
%%time

for i in range(len(rooms)):
    for j in range(i + 1, len(rooms)):
        cruns = candidate_wall_runs(face_walls, scaled_room_mask, inv_room_mask, i + 1, j + 1)
        if cruns:
            print(j + 1, i + 1, cruns)

2 1 [(90, 82, 28, 'h')]
6 1 [(67, 80, 22, 'v')]
4 2 [(100, 80, 16, 'v')]
6 2 [(91, 80, 7, 'v')]
7 2 [(117, 82, 28, 'h')]
6 3 [(66, 34, 46, 'h')]
7 4 [(117, 34, 46, 'h')]
9 8 [(66, 118, 46, 'h')]
CPU times: user 229 ms, sys: 6.14 ms, total: 235 ms
Wall time: 60.5 ms


In [319]:
%%time
candidate_wall_runs(face_walls, scaled_room_mask, inv_room_mask, 9, 8)

CPU times: user 7.15 ms, sys: 0 ns, total: 7.15 ms
Wall time: 3.57 ms


[(66, 118, 46, 'h')]